In [6]:
# Import library
import pandas as pd
import numpy as np

In [7]:
## Baca file dataset USDA
print("Membaca file dataset...")
food = pd.read_csv('food.csv')
food_nutrient = pd.read_csv('food_nutrient.csv')
nutrient = pd.read_csv('nutrient.csv')
food_portion = pd.read_csv('food_portion.csv')

Membaca file dataset...


C:\Users\ACER\AppData\Local\Temp\ipykernel_31576\896480988.py:4: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  food_nutrient = pd.read_csv('food_nutrient.csv')


In [8]:
## Cek data awal dan struktur
print("Memeriksa struktur data...")
print("Kolom food:", food.columns.tolist())
print("Kolom food_nutrient:", food_nutrient.columns.tolist())
print("Kolom food_portion:", food_portion.columns.tolist())

Memeriksa struktur data...
Kolom food: ['fdc_id', 'data_type', 'description', 'food_category_id', 'publication_date']
Kolom food_nutrient: ['id', 'fdc_id', 'nutrient_id', 'amount', 'data_points', 'derivation_id', 'min', 'max', 'median', 'footnote', 'min_year_acquired']
Kolom food_portion: ['id', 'fdc_id', 'seq_num', 'amount', 'measure_unit_id', 'portion_description', 'modifier', 'gram_weight', 'data_points', 'footnote', 'min_year_acquired']


In [9]:
## Cek nama nutrisi yang tersedia
print("\nMenampilkan 20 nutrisi pertama:")
print(nutrient[['id', 'name']].head(20))


Menampilkan 20 nutrisi pertama:
      id                                  name
0   2047      Energy (Atwater General Factors)
1   2048     Energy (Atwater Specific Factors)
2   1001                                Solids
3   1002                              Nitrogen
4   1003                               Protein
5   1004                     Total lipid (fat)
6   1005           Carbohydrate, by difference
7   1006  Fiber, crude (DO NOT USE - Archived)
8   1007                                   Ash
9   1008                                Energy
10  1009                                Starch
11  1010                               Sucrose
12  1011                               Glucose
13  1012                              Fructose
14  1013                               Lactose
15  1014                               Maltose
16  1015                               Amylose
17  1016                           Amylopectin
18  1017                                Pectin
19  1018                   

In [10]:
## Tentukan nutrisi yang dibutuhkan untuk aplikasi diet
# Cari nutrisi yang tersedia dengan nama yang mirip
print("\nMencari nutrisi kalori:")
print(nutrient[nutrient['name'].str.contains('Energy|Calorie', case=False)].head())
print("\nMencari nutrisi protein:")
print(nutrient[nutrient['name'].str.contains('Protein', case=False)].head())
print("\nMencari nutrisi lemak:")
print(nutrient[nutrient['name'].str.contains('Fat|Lipid', case=False)].head())
print("\nMencari nutrisi karbohidrat:")
print(nutrient[nutrient['name'].str.contains('Carbohydrate', case=False)].head())
print("\nMencari nutrisi serat:")
print(nutrient[nutrient['name'].str.contains('Fiber', case=False)].head())
print("\nMencari nutrisi gula:")
print(nutrient[nutrient['name'].str.contains('Sugar', case=False)].head())
print("\nMencari nutrisi sodium:")
print(nutrient[nutrient['name'].str.contains('Sodium', case=False)].head())


Mencari nutrisi kalori:
      id                               name unit_name  nutrient_nbr   rank
0   2047   Energy (Atwater General Factors)      KCAL         957.0  280.0
1   2048  Energy (Atwater Specific Factors)      KCAL         958.0  290.0
9   1008                             Energy      KCAL         208.0  300.0
63  1062                             Energy        kJ         268.0  400.0

Mencari nutrisi protein:
      id              name unit_name  nutrient_nbr   rank
4   1003           Protein         G         203.0  600.0
54  1053  Adjusted Protein         G         257.0  700.0

Mencari nutrisi lemak:
       id               name unit_name  nutrient_nbr      rank
5    1004  Total lipid (fat)         G         204.0     800.0
50   1049    Solids, non-fat         G         253.0  999999.0
86   1085   Total fat (NLEA)         G         298.0     900.0
256  1255      Phospholipids         G         603.0  999999.0
257  1256        Glycolipids         G         604.0  999999.

In [11]:
## Tentukan ID nutrisi yang dibutuhkan (gunakan nama yang tersedia di dataset)
# Sesuaikan dengan hasil pencarian di atas
nutrient_needed = [
    'Energy',                          # atau 'Calories, total'
    'Protein',                         # atau 'Protein, total'
    'Total lipid (fat)',               # atau 'Fat, total'
    'Carbohydrate, by difference',     # atau 'Carbohydrate, total'
    'Fiber, total dietary',            # atau 'Dietary fiber'
    'Sugars, total',                   # atau nama lain yang tersedia
    'Sodium, Na'                       # atau 'Sodium'
]

# Cari ID untuk nutrisi yang dibutuhkan dan tampilkan
needed_nutrient_info = nutrient[nutrient['name'].isin(nutrient_needed)]
print("\nNutrisi yang akan digunakan:")
print(needed_nutrient_info[['id', 'name']])
needed_nutrient_ids = needed_nutrient_info['id'].tolist()


Nutrisi yang akan digunakan:
      id                         name
4   1003                      Protein
5   1004            Total lipid (fat)
6   1005  Carbohydrate, by difference
9   1008                       Energy
63  1062                       Energy
80  1079         Fiber, total dietary
94  1093                   Sodium, Na


In [12]:
## Filter data nutrisi yang dibutuhkan
print("\nMemfilter data nutrisi...")
food_nutrient_filtered = food_nutrient[food_nutrient['nutrient_id'].isin(needed_nutrient_ids)]


Memfilter data nutrisi...


In [13]:
## Pivot data (1 makanan = 1 baris dengan semua nutrisi)
print("Melakukan pivot data...")
try:
    # Gunakan try-except untuk menangani kesalahan
    pivot = food_nutrient_filtered.pivot_table(
        index='fdc_id', 
        columns='nutrient_id', 
        values='amount',
        aggfunc='first'  # Ambil nilai pertama jika ada duplikat
    ).reset_index()
    
    # Buat dictionary untuk rename kolom pivot
    nutrient_dict = nutrient.set_index('id')['name'].to_dict()
    pivot = pivot.rename(columns=nutrient_dict)
    
    # Tampilkan hasil pivot
    print("Hasil pivot (5 baris pertama):")
    print(pivot.head())
except KeyError as e:
    print(f"Error saat pivot: {e}")
    # Tampilkan kolom yang ada di food_nutrient
    print("Kolom yang tersedia di food_nutrient:", food_nutrient.columns.tolist())
    # Jika 'fdc_id' tidak ada, cari kolom ID makanan
    if 'fdc_id' not in food_nutrient.columns:
        print("Kolom 'fdc_id' tidak ditemukan. Mencari kolom ID alternatif...")
        id_columns = [col for col in food_nutrient.columns if 'id' in col.lower()]
        print("Kolom ID yang ditemukan:", id_columns)
        # Gunakan kolom ID pertama yang ditemukan
        if id_columns:
            food_id_column = id_columns[0]
            print(f"Menggunakan '{food_id_column}' sebagai ID makanan")
            # Ulangi pivot dengan kolom ID yang benar
            pivot = food_nutrient_filtered.pivot_table(
                index=food_id_column, 
                columns='nutrient_id', 
                values='amount',
                aggfunc='first'
            ).reset_index()
            pivot = pivot.rename(columns=nutrient_dict)
    else:
        print("Terjadi kesalahan yang tidak dapat diatasi. Periksa struktur data.")
        raise

Melakukan pivot data...
Hasil pivot (5 baris pertama):
nutrient_id  fdc_id  Protein  Total lipid (fat)  Carbohydrate, by difference  \
0            319877      NaN               19.0                          NaN   
1            319878      NaN                NaN                          NaN   
2            319882      NaN               18.7                          NaN   
3            319884      NaN                NaN                          NaN   
4            319892      NaN               16.6                          NaN   

nutrient_id  Energy  Energy  Fiber, total dietary  Sodium, Na  
0               NaN     NaN                   NaN         NaN  
1               NaN     NaN                   NaN       446.0  
2               NaN     NaN                   NaN         NaN  
3               NaN     NaN                   NaN       444.0  
4               NaN     NaN                   NaN         NaN  


In [14]:
## Gabungkan dengan nama makanan
print("\nMenggabungkan dengan data makanan...")
try:
    # Cek kolom di food yang berisi ID dan deskripsi
    print("Kolom di food:", food.columns.tolist())
    
    # Cari kolom yang berisi ID makanan dan deskripsi
    food_id_column = 'fdc_id' if 'fdc_id' in food.columns else 'id'
    description_column = 'description' if 'description' in food.columns else 'name'
    
    # Gabungkan dengan data makanan
    data = pivot.merge(
        food[[food_id_column, description_column]], 
        left_on='fdc_id', 
        right_on=food_id_column
    )
    
    # Rename kolom
    data = data.rename(columns={description_column: 'name'})
    
    print("Hasil penggabungan (5 baris pertama):")
    print(data.head())
except Exception as e:
    print(f"Error saat merge dengan food: {e}")
    raise


Menggabungkan dengan data makanan...
Kolom di food: ['fdc_id', 'data_type', 'description', 'food_category_id', 'publication_date']
Hasil penggabungan (5 baris pertama):
   fdc_id  Protein  Total lipid (fat)  Carbohydrate, by difference  Energy  \
0  319877      NaN               19.0                          NaN     NaN   
1  319878      NaN                NaN                          NaN     NaN   
2  319882      NaN               18.7                          NaN     NaN   
3  319884      NaN                NaN                          NaN     NaN   
4  319892      NaN               16.6                          NaN     NaN   

   Energy  Fiber, total dietary  Sodium, Na    name  
0     NaN                   NaN         NaN  Hummus  
1     NaN                   NaN       446.0  Hummus  
2     NaN                   NaN         NaN  Hummus  
3     NaN                   NaN       444.0  Hummus  
4     NaN                   NaN         NaN  Hummus  


In [15]:
## Gabungkan dengan ukuran porsi
print("\nMenggabungkan dengan data porsi...")
try:
    # Cek kolom di food_portion
    print("Kolom di food_portion:", food_portion.columns.tolist())
    
    # Cari kolom yang berisi ID makanan, gram, dan deskripsi porsi
    portion_id_column = 'fdc_id' if 'fdc_id' in food_portion.columns else 'food_id'
    gram_column = 'gram_weight' if 'gram_weight' in food_portion.columns else 'amount'
    portion_desc_column = 'portion_description' if 'portion_description' in food_portion.columns else 'description'
    
    # Group by untuk mendapatkan satu porsi per makanan
    food_portion_simple = food_portion.groupby(portion_id_column).first().reset_index()
    
    # Gabungkan dengan data porsi
    data = data.merge(
        food_portion_simple[[portion_id_column, gram_column, portion_desc_column]],
        left_on='fdc_id',
        right_on=portion_id_column,
        how='left'
    )
    
    print("Hasil setelah penggabungan porsi (5 baris pertama):")
    print(data.head())
except Exception as e:
    print(f"Error saat merge dengan food_portion: {e}")
    # Jika gagal, tambahkan kolom porsi default
    print("Menambahkan kolom porsi default...")
    data['gram_weight'] = 100  # Default 100 gram
    data['portion_description'] = 'Default portion (100g)'


Menggabungkan dengan data porsi...
Kolom di food_portion: ['id', 'fdc_id', 'seq_num', 'amount', 'measure_unit_id', 'portion_description', 'modifier', 'gram_weight', 'data_points', 'footnote', 'min_year_acquired']
Hasil setelah penggabungan porsi (5 baris pertama):
   fdc_id  Protein  Total lipid (fat)  Carbohydrate, by difference  Energy  \
0  319877      NaN               19.0                          NaN     NaN   
1  319878      NaN                NaN                          NaN     NaN   
2  319882      NaN               18.7                          NaN     NaN   
3  319884      NaN                NaN                          NaN     NaN   
4  319892      NaN               16.6                          NaN     NaN   

   Energy  Fiber, total dietary  Sodium, Na    name  gram_weight  \
0     NaN                   NaN         NaN  Hummus          NaN   
1     NaN                   NaN       446.0  Hummus          NaN   
2     NaN                   NaN         NaN  Hummus          

In [16]:
## Tambahkan kolom kategori makanan (opsional, jika ada)
try:
    if 'food_category_id' in food.columns:
        print("\nMenambahkan kategori makanan...")
        # Baca file kategori makanan jika ada
        food_category = pd.read_csv('food_category.csv')
        # Gabungkan dengan kategori
        data = data.merge(
            food_category[['id', 'description']],
            left_on='food_category_id',
            right_on='id',
            how='left'
        )
        data = data.rename(columns={'description_y': 'food_category'})
except Exception as e:
    print(f"Tidak bisa menambahkan kategori makanan: {e}")
    # Tambahkan kategori default
    data['food_category'] = 'Uncategorized'


Menambahkan kategori makanan...
Tidak bisa menambahkan kategori makanan: 'food_category_id'


In [17]:
## Susun kolom akhir dan rename
print("\nMenyusun kolom akhir...")
# Dapatkan kolom yang tersedia di data
available_columns = data.columns.tolist()
print("Kolom yang tersedia:", available_columns)

# Cari kolom nutrisi yang tersedia
energy_col = next((col for col in available_columns if 'Energy' in col or 'Calorie' in col), None)
protein_col = next((col for col in available_columns if 'Protein' in col), None)
fat_col = next((col for col in available_columns if 'lipid' in col.lower() or 'fat' in col.lower()), None)
carb_col = next((col for col in available_columns if 'Carbohydrate' in col), None)
fiber_col = next((col for col in available_columns if 'Fiber' in col), None)
sugar_col = next((col for col in available_columns if 'Sugar' in col), None)
sodium_col = next((col for col in available_columns if 'Sodium' in col), None)

# Buat list kolom untuk final dataset
final_columns = ['name']
if energy_col: final_columns.append(energy_col)
if protein_col: final_columns.append(protein_col)
if fat_col: final_columns.append(fat_col)
if carb_col: final_columns.append(carb_col)
if fiber_col: final_columns.append(fiber_col)
if sugar_col: final_columns.append(sugar_col)
if sodium_col: final_columns.append(sodium_col)

# Tambahkan kolom porsi
if 'gram_weight' in available_columns: final_columns.append('gram_weight')
if 'portion_description' in available_columns: final_columns.append('portion_description')

# Tambahkan kategori jika ada
if 'food_category' in available_columns: final_columns.append('food_category')

# Buat dataframe final
print("Kolom final yang akan digunakan:", final_columns)
final = data[final_columns].copy()

# Rename kolom ke format yang lebih sederhana
column_mapping = {
    energy_col: 'calories' if energy_col else None,
    protein_col: 'protein' if protein_col else None,
    fat_col: 'fat' if fat_col else None,
    carb_col: 'carbohydrate' if carb_col else None,
    fiber_col: 'fiber' if fiber_col else None,
    sugar_col: 'sugar' if sugar_col else None,
    sodium_col: 'sodium' if sodium_col else None,
    'gram_weight': 'portion_size',
    'portion_description': 'serving_unit'
}

# Hapus None dari mapping
column_mapping = {k: v for k, v in column_mapping.items() if k is not None}
final = final.rename(columns=column_mapping)

# Ganti NaN dengan 0 untuk nilai nutrisi
for col in ['calories', 'protein', 'fat', 'carbohydrate', 'fiber', 'sugar', 'sodium']:
    if col in final.columns:
        final[col] = final[col].fillna(0)

# Tambahkan kolom food_category jika belum ada
if 'food_category' not in final.columns:
    final['food_category'] = 'Uncategorized'

# Tambahkan kolom allergens kosong
final['allergens'] = 'None'

# Tambahkan kolom glycemic_index default
final['glycemic_index'] = 0  # Default value


Menyusun kolom akhir...
Kolom yang tersedia: ['fdc_id', 'Protein', 'Total lipid (fat)', 'Carbohydrate, by difference', 'Energy', 'Energy', 'Fiber, total dietary', 'Sodium, Na', 'name', 'gram_weight', 'portion_description', 'food_category']
Kolom final yang akan digunakan: ['name', 'Energy', 'Protein', 'Total lipid (fat)', 'Carbohydrate, by difference', 'Fiber, total dietary', 'Sodium, Na', 'gram_weight', 'portion_description', 'food_category']


In [18]:
## Simpan ke CSV
print("\nMenyimpan hasil ke CSV...")
final.to_csv('usda_cleaned_for_diet_app.csv', index=False)
print("CSV cleaned berhasil disimpan!")


Menyimpan hasil ke CSV...
CSV cleaned berhasil disimpan!


In [19]:
## Lihat hasil akhir
print("\nHasil akhir (5 baris pertama):")
print(final.head())
print("\nInfo kolom:")
print(final.info())
print("\nStatistik deskriptif:")
print(final.describe())

# Tambahkan informasi jumlah makanan
print(f"\nTotal {len(final)} makanan berhasil diproses")


Hasil akhir (5 baris pertama):
     name  calories  calories  protein   fat  carbohydrate  fiber  sodium  \
0  Hummus       0.0       0.0      0.0  19.0           0.0    0.0     0.0   
1  Hummus       0.0       0.0      0.0   0.0           0.0    0.0   446.0   
2  Hummus       0.0       0.0      0.0  18.7           0.0    0.0     0.0   
3  Hummus       0.0       0.0      0.0   0.0           0.0    0.0   444.0   
4  Hummus       0.0       0.0      0.0  16.6           0.0    0.0     0.0   

   portion_size serving_unit  food_category allergens  glycemic_index  
0           NaN          NaN  Uncategorized      None               0  
1           NaN          NaN  Uncategorized      None               0  
2           NaN          NaN  Uncategorized      None               0  
3           NaN          NaN  Uncategorized      None               0  
4           NaN          NaN  Uncategorized      None               0  

Info kolom:


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9006 entries, 0 to 9005
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            9006 non-null   object 
 1   calories        9006 non-null   float64
 2   calories        9006 non-null   float64
 3   protein         9006 non-null   float64
 4   fat             9006 non-null   float64
 5   carbohydrate    9006 non-null   float64
 6   fiber           9006 non-null   float64
 7   sodium          9006 non-null   float64
 8   portion_size    112 non-null    float64
 9   serving_unit    1 non-null      object 
 10  food_category   9006 non-null   object 
 11  allergens       9006 non-null   object 
 12  glycemic_index  9006 non-null   int64  
dtypes: float64(8), int64(1), object(4)
memory usage: 914.8+ KB
None

Statistik deskriptif:
          calories     calories      protein          fat  carbohydrate  \
count  9006.000000  9006.000000  9006.000000  9006.